In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import datetime as dt

import numpy as np
import pandas as pd

from deep_hedging import (
    Ticker,
    Underlyings,
    Frequency,
    NelsonSiegelCurve,
    ZeroCouponBond,
    FixedCouponBond,
)
from deep_hedging.non_linear.exotic.basket import WorstOfDigitalCall
from deep_hedging.non_linear.exotic.quanto import QuantoOption

RANDOM_SEED = 12
PATH = Path("../../data/")

In [3]:
FIXED_PAYMENT = 0.005
DIGITAL_COUPON = 5.0
PAYMENT_BARRIER = 1.0
PAYMENT_FREQUENCY = Frequency.SEMIANNUALLY

FIXED_FINAL_PAYMENT = 1.0

N_YEARS_LOOKBACK = 5
FIXING = dt.datetime(2021, 4, 19)
FINAL_FIXING = dt.datetime(2024, 4, 17)

In [4]:
underlyings = Underlyings(
    tickers=[
        Ticker("Sanofi", "SAN.PA", currency="EUR"),
        Ticker("GlaxoSmithKline PLC", "GSK.L", currency="GBP"),
        Ticker("Bayer AG", "BAYN.DE", currency="EUR"),
    ],
    start=N_YEARS_LOOKBACK,
    end=FIXING,
)

[*********************100%%**********************]  3 of 3 completed


In [5]:
fx = Underlyings(
    tickers=[
        Ticker("EURRUB", "EURRUB=X", currency="EUR"),
        Ticker("GBPRUB", "GBPRUB=X", currency="GBP"),
    ],
    start=N_YEARS_LOOKBACK,
    end=FIXING,
)

# Issue with yahoo finance data - strange GBPRUB quote
fx.data = fx.data[fx["GBPRUB=X"] > 1.2]
fx.data

[*********************100%%**********************]  2 of 2 completed


Ticker,EURRUB=X,GBPRUB=X
Date,,
2004-02-13,36.470001,53.591999
2004-04-01,34.990002,52.884998
2004-07-27,35.070000,52.995998
2005-01-17,36.619999,52.074001
2005-02-22,36.689999,53.369999
...,...,...
2021-04-12,92.128601,106.120071
2021-04-13,92.092499,106.289673
2021-04-14,90.795601,104.455833


In [6]:
nss_params = pd.read_csv(PATH / "rub_nss.csv", sep=";")
rub_params = nss_params[
    nss_params["tradedate"] == f"{FIXING.strftime('%d.%m.%Y')}"
].loc[:, ["B1", "B2", "B3", "T1"]]
rub_params = rub_params.replace(",", ".", regex=True).astype(float)
rub_params

,B1,B2,B3,T1
1833,744.953996,-273.365215,-271.811557,0.905105


In [7]:
curve = NelsonSiegelCurve(
    b0=rub_params.B1.values[0] / 100,
    b1=rub_params.B2.values[0] / 100,
    b2=rub_params.B3.values[0] / 100,
    tau=rub_params.T1.values[0] / 100,
)

In [15]:
fixed_final_payment = (
    ZeroCouponBond(
        yield_curve=curve,
        start_date=FIXING,
        end_date=FINAL_FIXING,
    )
    * FIXED_FINAL_PAYMENT
)

worst_of_digital = WorstOfDigitalCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=PAYMENT_BARRIER,
    frequency=PAYMENT_FREQUENCY,
    start_date=FIXING,
    end_date=FINAL_FIXING,
    digital_coupon=DIGITAL_COUPON,
    random_seed=RANDOM_SEED,
)

warrant = worst_of_digital + fixed_final_payment
print(f"{warrant.price():,.4f}")

5.1744


In [16]:
fixed_bond = FixedCouponBond(
    yield_curve=curve,
    start_date=FIXING,
    end_date=FINAL_FIXING,
    fixed_coupon=FIXED_PAYMENT,
    frequency=PAYMENT_FREQUENCY,
)

worst_of_digital = WorstOfDigitalCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=PAYMENT_BARRIER,
    frequency=PAYMENT_FREQUENCY,
    start_date=FIXING,
    end_date=FINAL_FIXING,
    digital_coupon=DIGITAL_COUPON,
    random_seed=RANDOM_SEED,
)

warrant = fixed_bond + worst_of_digital
print(f"{warrant.price():,.4f}")

5.2008


In [12]:
fixed_bond = FixedCouponBond(
    yield_curve=curve,
    start_date=FIXING,
    end_date=FINAL_FIXING,
    fixed_coupon=FIXED_PAYMENT,
    frequency=PAYMENT_FREQUENCY,
)
fixed_bond

StructuredNote of:
* LONG 1.0 units of RiskFreeBond:
* Term = 3.0 years
* YTM = 7.43%
* Start Date = 2021-04-19 00:00:00
* End Date = 2024-04-17 00:00:00

* LONG 1.0 units of StructuredNote of:
* LONG 0.005 units of RiskFreeBond:
* Term = 0.5 years
* YTM = 7.35%
* Start Date = 2021-04-19 00:00:00
* End Date = 2021-10-18 00:00:00


* LONG 1.0 units of StructuredNote of:
* LONG 0.005 units of RiskFreeBond:
* Term = 1.0 years
* YTM = 7.4%
* Start Date = 2021-04-19 00:00:00
* End Date = 2022-04-19 00:00:00


* LONG 1.0 units of StructuredNote of:
* LONG 0.005 units of RiskFreeBond:
* Term = 1.5 years
* YTM = 7.42%
* Start Date = 2021-04-19 00:00:00
* End Date = 2022-10-19 00:00:00


* LONG 1.0 units of StructuredNote of:
* LONG 0.005 units of RiskFreeBond:
* Term = 2.0 years
* YTM = 7.43%
* Start Date = 2021-04-19 00:00:00
* End Date = 2023-04-19 00:00:00


* LONG 1.0 units of StructuredNote of:
* LONG 0.005 units of RiskFreeBond:
* Term = 2.5 years
* YTM = 7.43%
* Start Date = 2021-04-19 

In [13]:
fixed_bond.price()

0.8245341502202985

In [11]:
quanto = QuantoOption(
    option=worst_of_digital,
    modifying_underlyings=fx,
    yield_curve=curve,
    random_seed=RANDOM_SEED,
)
price = quanto.price(spot=[1.0] * (2 * len(underlyings)))
print(f"{100 * price:,.4f} RUB")

AssertionError: 